In [1]:
"""Heatmap."""

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from scipy.sparse import load_npz
from niceview.utils.tools import txt_to_list, select_col_from_name, normalize_array

In [2]:
def heatmap():
    """Plot heatmap.

    Args:
        scores (np.ndarray): 1D array of scores of shape (N,). 
        coords (np.ndarray): 2D array of coordinates of shape (N, 2).
        patch_size (int): predefined size of the patch.
    """
    # image size in (width x, height y)
    # patch size
    # coordinates of the patch centers
    # normalize scores
    # heatmap overlay: tracks attention score over each pixel of heatmap
    # overlay counter: tracks number of times each pixel is visited
    # average heatmap: heatmap overlay / overlay counter (only for non-zero counter)
    # blur heatmap
    pass

In [3]:
img_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-wsi-img.tiff'
spot_info_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-spot-info.csv'
spot_gene_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-spot-gene.npz'
spot_gene_name_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-spot-gene-name.txt'
selected_spot_gene_name = 'ENSG00000065534'

In [4]:
img = cv2.imread(img_path)
height, width, _ = img.shape
patch_size = (128, 128)
factor = 2
threshold = 2

In [5]:
patch_size  = np.ceil(np.array(patch_size)).astype(int) * factor

In [6]:
spot_info = pd.read_csv(spot_info_path)
spot_pos = spot_info[['x', 'y']].values
spot_diameter = spot_info['diameter'].values

spot_gene = load_npz(spot_gene_path)
spot_gene_name = txt_to_list(spot_gene_name_path)
spot_selected_gene = select_col_from_name(
    spot_gene, spot_gene_name, selected_spot_gene_name,
)
spot_selected_gene_norm = normalize_array(spot_selected_gene, 1, 255)
spot_selected_gene_norm = np.array(spot_selected_gene_norm).ravel()

In [7]:
spot_pos = spot_pos.astype(int)

In [8]:
overlay = np.full((height, width), 0).astype(float)
counter = np.full((height, width), 0).astype(np.uint16) 
for idx in range(len(spot_pos)):
    score = spot_selected_gene_norm[idx]
    # if score >= threshold:
    #     pass
    # else:
    #     print(f'Low score: {score}')
    #     score = 0.0
    coord = spot_pos[idx]  # coord in (x, y)
    
    overlay[coord[1]: coord[1] + patch_size[1], coord[0]: coord[0] + patch_size[0]] += score
    counter[coord[1]: coord[1] + patch_size[1], coord[0]: coord[0] + patch_size[0]] += 1

In [9]:
zero_mask = counter == 0

In [10]:
overlay[~zero_mask] = overlay[~zero_mask] / counter[~zero_mask]

In [11]:
overlap = 0.5
kernel = tuple((patch_size * (1 - overlap)).astype(int) * 2 + 1)
print(kernel)

(257, 257)


In [12]:
mask = cv2.GaussianBlur(overlay, kernel, 0) 

In [13]:
mask

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
from niceview.utils.tools import mask_to_image

In [15]:
backtorgb = mask_to_image(mask, cmap=cv2.COLORMAP_JET)

In [16]:
cv2.imwrite('overlay.png', backtorgb)

True